In [ ]:
from keras.models import Sequential, Model, load_model
from keras.layers import UpSampling2D, Conv2D, Activation, BatchNormalization, Reshape, Dense, Input, LeakyReLU, Dropout, Flatten, ZeroPadding2D
from keras.optimizers import Adam

import glob
from PIL import Image
import numpy as np
import os
from ast import literal_eval

from scipy.misc import imsave
import xml.etree.ElementTree as ET 
from tqdm import tqdm
import zipfile
import time

In [ ]:
class DataLoader:
    def __init__(self, root='../input/', image_size=(64, 64)):
        self.root = root
        self.breeds = os.listdir(self.root + 'annotation/Annotation/')
        self.idxIn = 0
        self.namesIn = []
        self.image_size = image_size
        self.imagesIn = np.zeros((25000, image_size[0], image_size[1], 3))
    
    def load_images(self):
        start = time.time()
        for breed in self.breeds:
            for dog in os.listdir(self.root + 'annotation/Annotation/' + breed):
                try:
                    img = Image.open(self.root + 'all-dogs/all-dogs/' + dog + '.jpg')
                except:
                    continue
                tree = ET.parse(self.root + 'annotation/Annotation/' + breed + '/' + dog)
                root = tree.getroot()
                objects = root.findall('object')
                for o in objects:
                    bndbox = o.find('bndbox') 
                    xmin = int(bndbox.find('xmin').text)
                    ymin = int(bndbox.find('ymin').text)
                    xmax = int(bndbox.find('xmax').text)
                    ymax = int(bndbox.find('ymax').text)
                    w = np.min((xmax - xmin, ymax - ymin))
                    img2 = img.crop((xmin, ymin, xmin+w, ymin+w))
                    img2 = img2.resize((self.image_size[0], self.image_size[1]), Image.ANTIALIAS)
                    self.imagesIn[self.idxIn,:,:,:] = np.asarray(img2)
                    if self.idxIn % 1000 == 0:
                        print(self.idxIn)
                    self.namesIn.append(breed)
                    self.idxIn += 1
        idx = np.arange(self.idxIn)
        np.random.shuffle(idx)
        self.imagesIn = self.imagesIn[idx,:,:,:]
        self.namesIn = np.array(self.namesIn)[idx]

        end = time.time()
        print(end - start)

        return self.imagesIn

In [ ]:
class ImageZipper:
    def __init__(self, folder='./', name='images.zip'):
        self.folder = folder
        self.z = zipfile.PyZipFile(self.folder + name, mode='w')
    
    def add_image(self, image, image_name):
        file_name = self.folder + image_name + '.png'
        imsave(file_name, image)
        self.z.write(file_name)
        os.remove(file_name)
    
    def close(self):
        self.z.close()

In [ ]:
class DCGAN:
    def __init__(self, discriminator_path, generator_path, output_directory, data_loader):
        self.data_loader = data_loader
        self.img_size = self.data_loader.image_size
        self.upsample_layers = 5
        self.starting_filters = 64
        self.kernel_size = 3
        self.channels = 3
        self.discriminator_path = discriminator_path
        self.generator_path = generator_path
        self.output_directory = output_directory
        self.generated_folder = f"{self.output_directory}/generated_{self.img_size[0]}x{self.img_size[1]}"

    def build_generator(self):
        noise_shape = (100,)

        # This block of code can be a little daunting, but essentially it automatically calculates the required starting
        # array size that will be correctly upscaled to our desired image size.
        #
        # We have 5 Upsample2D layers which each double the images width and height, so we can determine the starting
        # x size by taking (x / 2^upsample_count) So for our target image size, 256x192, we do the following:
        # x = (192 / 2^5), y = (256 / 2^5) [x and y are reversed within the model]
        # We also need a 3rd dimension which is chosen relatively arbitrarily, in this case it's 64.
        model = Sequential()
        model.add(
            Dense(self.starting_filters * (self.img_size[0] // (2 ** self.upsample_layers))  *  (self.img_size[1] // (2 ** self.upsample_layers)),
                  activation="relu", input_shape=noise_shape))
        model.add(Reshape(((self.img_size[0] // (2 ** self.upsample_layers)),
                           (self.img_size[1] // (2 ** self.upsample_layers)),
                           self.starting_filters)))
        model.add(BatchNormalization(momentum=0.8))

        model.add(UpSampling2D())  # 6x8 -> 12x16
        model.add(Conv2D(1024, kernel_size=self.kernel_size, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))

        model.add(UpSampling2D())  # 12x16 -> 24x32
        model.add(Conv2D(512, kernel_size=self.kernel_size, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))

        model.add(UpSampling2D())  # 24x32 -> 48x64
        model.add(Conv2D(256, kernel_size=self.kernel_size, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))

        model.add(UpSampling2D())  # 48x64 -> 96x128
        model.add(Conv2D(128, kernel_size=self.kernel_size, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))

        model.add(UpSampling2D())  # 96x128 -> 192x256
        model.add(Conv2D(64, kernel_size=self.kernel_size, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2D(32, kernel_size=self.kernel_size, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2D(self.channels, kernel_size=self.kernel_size, padding="same"))
        model.add(Activation("tanh"))

#         model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        img_shape = (self.img_size[0], self.img_size[1], self.channels)

        model = Sequential()

        model.add(Conv2D(32, kernel_size=self.kernel_size, strides=2, input_shape=img_shape, padding="same"))  # 192x256 -> 96x128
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Conv2D(64, kernel_size=self.kernel_size, strides=2, padding="same"))  # 96x128 -> 48x64
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2D(128, kernel_size=self.kernel_size, strides=2, padding="same"))  # 48x64 -> 24x32
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2D(256, kernel_size=self.kernel_size, strides=1, padding="same"))  # 24x32 -> 12x16
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Conv2D(512, kernel_size=self.kernel_size, strides=1, padding="same"))  # 12x16 -> 6x8
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

#         model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def build_gan(self):
        optimizer = Adam(0.0002, 0.5)

        # See if the specified model paths exist, if they don't then we start training new models

        if self.discriminator_path is not None and self.generator_path is not None and os.path.exists(self.discriminator_path) and os.path.exists(self.generator_path):
            self.discriminator = load_model(self.discriminator_path)
            self.generator = load_model(self.generator_path)
            print("Loaded models...")
        else:
            self.discriminator = self.build_discriminator()
            self.discriminator.compile(loss='binary_crossentropy',
                                       optimizer=optimizer,
                                       metrics=['accuracy'])

            self.generator = self.build_generator()
            self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # These next few lines setup the training for the GAN model
        z = Input(shape=(100,))
        img = self.generator(z)

        self.discriminator.trainable = False

        valid = self.discriminator(img)

        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def train(self, epochs, batch_size=32, save_interval=50):
        self.build_gan()
        X_train = self.data_loader.load_images()
        print("Training Data Shape: ", X_train.shape)

        # Rescale images from -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5

        half_batch = batch_size // 2

        for epoch in range(epochs):


            # Train Generator
            noise = np.random.normal(0, 1, (batch_size, 100))
            g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))



            # Train Discriminator
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            # Sample noise and generate a half batch of new images
            noise = np.random.normal(0, 1, (half_batch, 100))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # If at save interval => save generated image samples, save model files
            if epoch % (save_interval) == 0:
                
                # Print progress
                print(f"{epoch} [D loss: {d_loss[0]} | D Accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

#                 save_path = self.output_directory
#                 if not os.path.exists(save_path):
#                     os.makedirs(save_path)
#                 self.discriminator.save(save_path + "/discrim.h5")
#                 self.generator.save(save_path + "/generat.h5")

    def gene_imgs(self, count):
        # Generate images from the currently loaded model
        noise = np.random.normal(0, 1, (count, 100))
        return self.generator.predict(noise)
    
    def generate_imgs(self, count, threshold):
        # Generates (count) images from the model ensuring the discriminator scores them between the threshold values
        # and saves them

        imgs = []
        for i in tqdm(range(count), desc='Generating images'):
            score = [0]
            while not(threshold[0] < score[0] < threshold[1]):
                img = self.gene_imgs(1)
                score = self.discriminator.predict(img)
            imgs.append(img)

        imgs = np.asarray(imgs).squeeze()
        imgs = 0.5 * imgs + 0.5

        return imgs

In [ ]:
# Number of epochs to train for
epochs = 530000
# Number of images to train on at once
batch_size = 24
# How many epochs to go between saves/outputs
save_interval = 1000
# Directoy to save weights and images to.
output_directory = './'

data_loader = DataLoader()
dcgan = DCGAN(None, None, output_directory, data_loader)
start = time.time()
dcgan.train(epochs=epochs, batch_size=batch_size, save_interval=save_interval)
end = time.time()
print(end - start)

In [ ]:
# Path to existing generator weights file
load_generator = "./generat.h5"
# Path to existing discriminator weights file
load_discriminator = "./discrim.h5"

# generate that many samples from existing model
sample = 10000
# The values between which a generated image must score from the discriminator
sample_thresholds = (0.0, 1.0)
dcgan = DCGAN(load_discriminator, load_generator, output_directory, data_loader)
dcgan.build_gan()

zipper = ImageZipper(folder=output_directory)
for i in range(0, sample, 100):
    print('Images generated', i)
    images = dcgan.generate_imgs(100, sample_thresholds)
    for index, image in enumerate(images):
        zipper.add_image(image, str(i + index))

zipper.close()

In [ ]:
import os
os.listdir('.')